In [4]:
import numpy as np
from onnx import TensorProto, helper

import finn.core.onnx_exec as oxe
from finn.analysis.fpgadataflow.exp_cycles_per_layer import exp_cycles_per_layer
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.general.im2col import compute_conv_output_dim
from finn.custom_op.registry import getCustomOp
from finn.transformation.fpgadataflow.compile_cppsim import CompileCppSim
from finn.transformation.fpgadataflow.hlssynth_ip import HLSSynthIP
from finn.transformation.fpgadataflow.prepare_cppsim import PrepareCppSim
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP
from finn.transformation.fpgadataflow.prepare_rtlsim import PrepareRTLSim
from finn.transformation.fpgadataflow.set_exec_mode import SetExecMode
from finn.transformation.general import GiveUniqueNodeNames
from finn.util.basic import gen_finn_dt_tensor


def make_single_im2col_modelwrapper(
    k, ifm_ch, ifm_dim, ofm_dim, simd, stride, dilation, idt
):
    k_h, k_w = k
    ifm_dim_h, ifm_dim_w = ifm_dim
    stride_h, stride_w = stride
    dilation_h, dilation_w = dilation
    ofm_dim_h, ofm_dim_w = ofm_dim

    odt = idt
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_dim_h, ifm_dim_w, ifm_ch]
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_dim_h, ofm_dim_w, k_h * k_w * ifm_ch]
    )

    im2col_node = helper.make_node(
        "Im2Col",
        ["inp"],
        ["outp"],
        domain="finn.custom_op.general",
        stride=[stride_h, stride_w],
        kernel_size=[k_h, k_w],
        input_shape=str((1, ifm_dim_h, ifm_dim_w, ifm_ch)),
        dilations=[dilation_h, dilation_w],
        pad_amount=[0, 0, 0, 0],
        pad_value=0,
    )
    graph = helper.make_graph(
        nodes=[im2col_node], name="im2col_graph", inputs=[inp], outputs=[outp]
    )

    model = helper.make_model(graph, producer_name="im2col-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)

    return model


def make_single_slidingwindow_modelwrapper(
    k, ifm_ch, ifm_dim, ofm_dim, simd, stride, dilation, idt, dw=0
):
    k_h, k_w = k
    ifm_dim_h, ifm_dim_w = ifm_dim
    stride_h, stride_w = stride
    dilation_h, dilation_w = dilation
    ofm_dim_h, ofm_dim_w = ofm_dim

    odt = idt
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_dim_h, ifm_dim_w, ifm_ch]
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_dim_h, ofm_dim_w, k_h * k_w * ifm_ch]
    )

    SlidingWindow_node = helper.make_node(
        "ConvolutionInputGenerator1D",
        ["inp"],
        ["outp"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        ConvKernelDim=[k_h, k_w],
        IFMChannels=ifm_ch,
        IFMDim=[ifm_dim_h, ifm_dim_w],
        OFMDim=[ofm_dim_h, ofm_dim_w],
        SIMD=simd,
        Stride=[stride_h, stride_w],
        Dilation=[dilation_h, dilation_w],
        inputDataType=idt.name,
        outputDataType=odt.name,
        depthwise=dw,
    )
    graph = helper.make_graph(
        nodes=[SlidingWindow_node],
        name="slidingwindow_graph",
        inputs=[inp],
        outputs=[outp],
    )

    model = helper.make_model(graph, producer_name="slidingwindow-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)
    model.set_tensor_datatype("outp", odt)

    return model


def prepare_inputs(input_tensor):
    return {"inp": input_tensor}


def test_fpgadataflow_slidingwindow_1d(
    idt, k, ifm_dim, ifm_ch, stride, dilation, exec_mode, simd, dw, flip
):
    if flip:
        k = k[::-1]
        ifm_dim = ifm_dim[::-1]
        stride = stride[::-1]
        dilation = dilation[::-1]

    k_h, k_w = k
    ifm_dim_h, ifm_dim_w = ifm_dim
    stride_h, stride_w = stride
    dilation_h, dilation_w = dilation

    if (dilation_h > 1 or dilation_w > 1) and (stride_h > 1 or stride_w > 1):
        pytest.skip(
            """Dilation value greater than 1 and stride greater than 1
            currently not supported for 1D convolutions"""
        )
    if simd > ifm_ch:
        pytest.skip("SIMD cannot be larger than number of input channels")

    ofm_dim_h = compute_conv_output_dim(ifm_dim_h, k_h, stride_h, 0, dilation_h)
    ofm_dim_w = compute_conv_output_dim(ifm_dim_w, k_w, stride_w, 0, dilation_w)
    ofm_dim = [ofm_dim_h, ofm_dim_w]

    x = gen_finn_dt_tensor(idt, (1, ifm_dim_h, ifm_dim_w, ifm_ch))
    model = make_single_slidingwindow_modelwrapper(
        k=k,
        ifm_ch=ifm_ch,
        ifm_dim=ifm_dim,
        ofm_dim=ofm_dim,
        simd=simd,
        stride=stride,
        dilation=dilation,
        idt=idt,
        dw=dw,
    )

    if exec_mode == "cppsim":
        model = model.transform(SetExecMode("cppsim"))
        model = model.transform(PrepareCppSim())
        model = model.transform(CompileCppSim())
    elif exec_mode == "rtlsim":
        model = model.transform(SetExecMode("rtlsim"))
        model = model.transform(GiveUniqueNodeNames())
        model = model.transform(PrepareIP("xc7z020clg400-1", 5))
        model = model.transform(HLSSynthIP())
        model = model.transform(PrepareRTLSim())
    else:
        raise Exception("Unknown exec_mode in test_fpgadataflow_slidingwindow")

    # prepare input data
    input_dict = prepare_inputs(x)
    # execute model
    y_produced = oxe.execute_onnx(model, input_dict)["outp"]
    golden = make_single_im2col_modelwrapper(
        k=k,
        ifm_ch=ifm_ch,
        ifm_dim=ifm_dim,
        ofm_dim=ofm_dim,
        simd=simd,
        stride=stride,
        dilation=dilation,
        idt=idt,
    )
    y_expected = oxe.execute_onnx(golden, input_dict)["outp"]

    if dw == 0:
        return x, y_produced, y_expected
        #assert (y_produced == y_expected).all()
    else:
        y_expected = y_expected.reshape(
            1, ofm_dim_h, ofm_dim_w, k_h * k_w, ifm_ch // simd, simd
        )
        y_expected = y_expected.transpose(0, 1, 2, 4, 3, 5)
        y_expected = y_expected.reshape(1, ofm_dim_h, ofm_dim_w, ifm_ch * k_h * k_w)
        return x, y_produced, y_expected
        #assert (y_produced == y_expected).all()

    if exec_mode == "rtlsim":
        node = model.get_nodes_by_op_type("ConvolutionInputGenerator1D")[0]
        inst = getCustomOp(node)
        cycles_rtlsim = inst.get_nodeattr("cycles_rtlsim")
        exp_cycles_dict = model.analysis(exp_cycles_per_layer)
        exp_cycles = exp_cycles_dict[node.name]
        assert np.isclose(exp_cycles, cycles_rtlsim, atol=10)
        assert exp_cycles != 0
        
    return x, y_produced, y_expected

In [5]:
idt = DataType.INT8
k = [1, 51]
ifm_dim = [1, 178]
ifm_ch = 512
stride = [1, 1]
dilation = [1, 1]
exec_mode = "cppsim"
simd = 256
dw = 1
flip = False

IFMDim_x = ifm_dim[1]
IFMChannels = ifm_ch

x = np.arange(1, IFMDim_x*IFMChannels+1).reshape((IFMChannels, IFMDim_x)).astype(np.float32) 
x = np.transpose(x)
x = x.reshape((1,1,IFMDim_x, IFMChannels))
        
#print(x)

x, y_p, y_e = test_fpgadataflow_slidingwindow_1d(idt, k, ifm_dim, ifm_ch, stride, dilation, exec_mode, simd, dw, flip)

In [6]:
import numpy as np

#print(x)
print(np.shape(x))

#print(y_e)

#print(y_p)
print(np.shape(y_p))

print(np.array_equal(y_e,y_p))


(1, 1, 178, 512)
(1, 1, 128, 26112)
True


In [12]:
IFMDim_x = 178
IFMChannels = 512
ConvKernelDim_x = 51
OFMDim_x = 128
Stride_x=1
SIMD = 64
Multi_Input=1
multiplying_factor = IFMChannels/SIMD

latency_v1 = IFMDim_x * multiplying_factor / Multi_Input + np.max([OFMDim_x*ConvKernelDim_x*multiplying_factor, Stride_x*IFMDim_x*multiplying_factor/Multi_Input])
latency_v2 =  multiplying_factor*(ConvKernelDim_x-1)-(ConvKernelDim_x-1) + (OFMDim_x * ConvKernelDim_x * multiplying_factor)
#latency_v3 = ConvKernelDim_x*multiplying_factor + (OFMDim_x * ConvKernelDim_x * multiplying_factor)

print("Latency case v1: {}".format(int(latency_v1)))
print("Latency case v2: {}".format(int(latency_v2)))
#print("Latency case v3: {}".format(int(latency_v3)))


Latency case v1: 53648
Latency case v2: 52574


# Simulation algorithm

In [5]:
#image = [[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20]]
#image_seq = [1,11,2,12,3,13,4,14,5,15,6,16,7,17,8,18,9,19,10,20]
IFMDim_x = 10
IFMChannels = 2
OFMDim_x = 8
ConvKernelDim_x = 3

#IFMDim_x = 10
#IFMChannels = 3

image = np.arange(1, IFMDim_x*IFMChannels+1).reshape((IFMChannels, IFMDim_x))
image_seq = []
for i in range(IFMDim_x):
    for j in range(IFMChannels):
        image_seq.append(image[j][i])

print(image)
print(image_seq)

[[[[ 1 11]
   [ 2 12]
   [ 3 13]
   [ 4 14]
   [ 5 15]
   [ 6 16]
   [ 7 17]
   [ 8 18]
   [ 9 19]
   [10 20]]]]
[1, 11, 2, 12, 3, 13, 4, 14, 5, 15, 6, 16, 7, 17, 8, 18, 9, 19, 10, 20]


In [85]:
import numpy as np


baseIter = IFMChannels + OFMDim_x * (ConvKernelDim_x * IFMChannels)

buffer = np.zeros(ConvKernelDim_x*IFMChannels)

current_line=0 # when to read image
index_out=0 # pointer to buffer to write output
j=0 # tracking whether we add ConvKernelDim_x or IFMChannels to index_out offset
internal_counter=0 
for i in range(0,baseIter):
    print("Cycle: {:3} --".format(i), end=" ")
    if current_line < IFMChannels: # fill initial buffer
        buffer[current_line] = image_seq[current_line]
        print("{:20} W({}) --".format(str(buffer), current_line), end=" ")
        current_line +=1
    else:
        # Read input buffer
        if current_line < 20:
            if i < IFMChannels + ConvKernelDim_x * IFMChannels:
                index = current_line % (ConvKernelDim_x*IFMChannels)
                buffer[index] = image_seq[current_line]
                str_print = str(buffer) + " W(" + str(index) + ")"
                print("{:30} --".format(str(str_print)), end=" ")
                current_line +=1
            else:
                if internal_counter >= (ConvKernelDim_x*IFMChannels-IFMChannels):
                    index = current_line % (ConvKernelDim_x*IFMChannels)
                    buffer[index] = image_seq[current_line]
                    str_print = str(buffer) + " W(" + str(index) + ")"
                    print("{:30} --".format(str(str_print)), end=" ")
                    current_line += 1
                    internal_counter += 1
                    if internal_counter==ConvKernelDim_x*IFMChannels:
                        internal_counter=0
                else:
                    print("{:30} --".format(str(buffer)), end=" ")
                    internal_counter+=1
        else:
            print("{:30} --".format(str(buffer)), end=" ")
            
        # Write to output
        print("Out: {} at id({})".format(int(buffer[index_out]), index_out), end="")
        
        # Update write index pointer
        if j < ConvKernelDim_x-1:
            index_out = index_out + IFMChannels
            j+=1
        else:
            index_out = index_out + ConvKernelDim_x
            j=0
        index_out = index_out%(ConvKernelDim_x*IFMChannels)
        
    print("\n", end="")


Cycle:   0 -- [1. 0. 0. 0. 0. 0.]  W(0) -- 
Cycle:   1 -- [ 1. 11.  0.  0.  0.  0.] W(1) -- 
Cycle:   2 -- [ 1. 11.  2.  0.  0.  0.] W(2) -- Out: 1 at id(0)
Cycle:   3 -- [ 1. 11.  2. 12.  0.  0.] W(3) -- Out: 2 at id(2)
Cycle:   4 -- [ 1. 11.  2. 12.  3.  0.] W(4) -- Out: 3 at id(4)
Cycle:   5 -- [ 1. 11.  2. 12.  3. 13.] W(5) -- Out: 11 at id(1)
Cycle:   6 -- [ 4. 11.  2. 12.  3. 13.] W(0) -- Out: 12 at id(3)
Cycle:   7 -- [ 4. 14.  2. 12.  3. 13.] W(1) -- Out: 13 at id(5)
Cycle:   8 -- [ 4. 14.  2. 12.  3. 13.]      -- Out: 2 at id(2)
Cycle:   9 -- [ 4. 14.  2. 12.  3. 13.]      -- Out: 3 at id(4)
Cycle:  10 -- [ 4. 14.  2. 12.  3. 13.]      -- Out: 4 at id(0)
Cycle:  11 -- [ 4. 14.  2. 12.  3. 13.]      -- Out: 12 at id(3)
Cycle:  12 -- [ 4. 14.  5. 12.  3. 13.] W(2) -- Out: 13 at id(5)
Cycle:  13 -- [ 4. 14.  5. 15.  3. 13.] W(3) -- Out: 14 at id(1)
Cycle:  14 -- [ 4. 14.  5. 15.  3. 13.]      -- Out: 3 at id(4)
Cycle:  15 -- [ 4. 14.  5. 15.  3. 13.]      -- Out: 4 at id(0)
Cycle

# Algorithm 2

In [180]:
#image = [[1,2,3,4,5,6,7,8,9,10],[11,12,13,14,15,16,17,18,19,20]]
#image_seq = [1,11,2,12,3,13,4,14,5,15,6,16,7,17,8,18,9,19,10,20]
IFMDim_x = 20
IFMChannels = 10
ConvKernelDim_x = 7
OFMDim_x = IFMDim_x-ConvKernelDim_x+1

#IFMDim_x = 10
#IFMChannels = 3

image = np.arange(1, IFMDim_x*IFMChannels+1).reshape((IFMChannels, IFMDim_x))
image_seq = []
for i in range(IFMDim_x):
    for j in range(IFMChannels):
        image_seq.append(image[j][i])

#print(image)
#print(image_seq)

In [177]:
import numpy as np


baseIter = IFMChannels*(ConvKernelDim_x-1)-(ConvKernelDim_x-1) + OFMDim_x * (ConvKernelDim_x * IFMChannels)

buffer = np.zeros(ConvKernelDim_x*IFMChannels)

current_line=0 # when to read image
index_out=0 # pointer to buffer to write output
j=0 # tracking whether we add ConvKernelDim_x or IFMChannels to index_out offset
internal_counter=0 

output = []

print("Image: 1 x {}".format(IFMDim_x))
print("Kernel: 1 x {}".format(ConvKernelDim_x))
print("Channels: {}".format(IFMChannels))
print("Output: 1 x {}".format(OFMDim_x))

for i in range(0,baseIter):
    print("Cycle: {:3} --".format(i), end=" ")
    if current_line < IFMChannels*(ConvKernelDim_x-1)-(ConvKernelDim_x-1): # fill initial buffer
        buffer[current_line] = image_seq[current_line]
        print("{:40} W({}) --".format(str(buffer), current_line), end=" ")
        current_line +=1
    else:
        # Read input buffer (while current_line indicates it has not read all pixels)
        if current_line < IFMDim_x*IFMChannels:
            if current_line < ConvKernelDim_x * IFMChannels:
                index = current_line % (ConvKernelDim_x*IFMChannels)
                buffer[index] = image_seq[current_line]
                str_print = str(buffer) + " W(" + str(index) + ")"
                print("{:40} --".format(str(str_print)), end=" ")
                current_line +=1
                internal_counter +=1
            else:
                if internal_counter >= (ConvKernelDim_x*IFMChannels-IFMChannels):
                    index = current_line % (ConvKernelDim_x*IFMChannels)
                    buffer[index] = image_seq[current_line]
                    str_print = str(buffer) + " W(" + str(index) + ")"
                    print("{:40} --".format(str(str_print)), end=" ")
                    current_line += 1
                    internal_counter += 1
                    if internal_counter==ConvKernelDim_x*IFMChannels:
                        internal_counter=0
                else:
                    print("{:40} --".format(str(buffer)), end=" ")
                    internal_counter+=1
        else:
            print("{:40} --".format(str(buffer)), end=" ")
            
        # Write to output
        str_print="Out: "+ str(int(buffer[index_out])) + " at id ("+str(index_out)+")"
        print("{:20}".format(str_print), end="")
        output.append(int(buffer[index_out]))
        
        # Update write index pointer
        if j < ConvKernelDim_x-1:
            index_out = index_out + IFMChannels
            j+=1
        else:
            index_out = index_out + (IFMChannels+1) #ConvKernelDim_x
            j=0
        index_out = index_out%(ConvKernelDim_x*IFMChannels)
        
    
    print("\n", end="")


Image: 1 x 10
Kernel: 1 x 3
Channels: 2
Output: 1 x 8
Cycle:   0 -- [1. 0. 0. 0. 0. 0.]                      W(0) -- 
Cycle:   1 -- [ 1. 11.  0.  0.  0.  0.]                W(1) -- 
Cycle:   2 -- [ 1. 11.  2.  0.  0.  0.] W(2)           -- Out: 1 at id (0)    
Cycle:   3 -- [ 1. 11.  2. 12.  0.  0.] W(3)           -- Out: 2 at id (2)    
Cycle:   4 -- [ 1. 11.  2. 12.  3.  0.] W(4)           -- Out: 3 at id (4)    
Cycle:   5 -- [ 1. 11.  2. 12.  3. 13.] W(5)           -- Out: 11 at id (1)   
Cycle:   6 -- [ 4. 11.  2. 12.  3. 13.] W(0)           -- Out: 12 at id (3)   
Cycle:   7 -- [ 4. 14.  2. 12.  3. 13.] W(1)           -- Out: 13 at id (5)   
Cycle:   8 -- [ 4. 14.  2. 12.  3. 13.]                -- Out: 2 at id (2)    
Cycle:   9 -- [ 4. 14.  2. 12.  3. 13.]                -- Out: 3 at id (4)    
Cycle:  10 -- [ 4. 14.  2. 12.  3. 13.]                -- Out: 4 at id (0)    
Cycle:  11 -- [ 4. 14.  2. 12.  3. 13.]                -- Out: 12 at id (3)   
Cycle:  12 -- [ 4. 14.  5. 1

In [182]:
import numpy as np
from onnx import TensorProto, helper

import finn.core.onnx_exec as oxe
from finn.core.datatype import DataType
from finn.core.modelwrapper import ModelWrapper
from finn.custom_op.general.im2col import compute_conv_output_dim
from finn.transformation.infer_datatypes import InferDataTypes
from finn.transformation.infer_shapes import InferShapes

def execution_im2col(x, idt, k_h, k_w, stride_h, stride_w, ifm_ch, ifm_dim_h, ifm_dim_w, pad_amt, pad_val=0,
                     dilation_h=1, dilation_w=1):
    pad_amt_h = pad_amt[0] + pad_amt[2]
    pad_amt_w = pad_amt[1] + pad_amt[3]
    ofm_dim_h = compute_conv_output_dim(ifm_dim_h, k_h, stride_h, pad_amt_h, dilation_h)
    ofm_dim_w = compute_conv_output_dim(ifm_dim_w, k_w, stride_w, pad_amt_w, dilation_w)

    # set up onnx model
    inp = helper.make_tensor_value_info(
        "inp", TensorProto.FLOAT, [1, ifm_dim_h, ifm_dim_w, ifm_ch]
    )
    outp = helper.make_tensor_value_info(
        "outp", TensorProto.FLOAT, [1, ofm_dim_h, ofm_dim_w, k_h * k_w * ifm_ch]
    )

    im2col_node = helper.make_node(
        "Im2Col",
        ["inp"],
        ["outp"],
        domain="finn.custom_op.general",
        stride=[stride_h, stride_w],
        kernel_size=[k_h, k_w],
        pad_amount=pad_amt,
        pad_value=pad_val,
        input_shape="(1,{},{},{})".format(ifm_dim_h, ifm_dim_w, ifm_ch),
        dilations=[dilation_h, dilation_w],
    )

    graph = helper.make_graph(
        nodes=[im2col_node], name="im2col_graph", inputs=[inp], outputs=[outp]
    )

    model = helper.make_model(graph, producer_name="im2col-model")
    model = ModelWrapper(model)

    model.set_tensor_datatype("inp", idt)

    # test shape inference
    model.transform(InferShapes())
    assert model.get_tensor_shape("outp") == [
        1,
        ofm_dim_h,
        ofm_dim_w,
        k_h * k_w * ifm_ch,
    ]

    # test datatype inference
    assert model.get_tensor_datatype("outp") is DataType.FLOAT32
    model = model.transform(InferDataTypes())
    assert model.get_tensor_datatype("outp") is idt

    # prepare input data
    input_dict = {"inp": x}

    # execute model
    y_produced = oxe.execute_onnx(model, input_dict)["outp"]

    return y_produced


x = image

x = np.transpose(x)

x = x.reshape((1,1,IFMDim_x, IFMChannels))

y_expected = execution_im2col(
    x=x,
    idt=DataType.INT4,
    k_h=1,
    k_w=ConvKernelDim_x,
    stride_h=1,
    stride_w=1,
    ifm_ch=IFMChannels,
    ifm_dim_h=1,
    ifm_dim_w=IFMDim_x,
    pad_amt=[0,0,0,0],
    pad_val=0,
    dilation_h=1,
    dilation_w=1,
)

y_expected = y_expected.reshape(1, 1, OFMDim_x, ConvKernelDim_x, IFMChannels, 1)
y_expected = y_expected.transpose(0,1,2,4,3,5)
y_expected = y_expected.reshape(1, 1, OFMDim_x, IFMChannels*ConvKernelDim_x)
y_expected = y_expected.reshape(OFMDim_x, IFMChannels*ConvKernelDim_x)
print(y_expected)

y_produced = np.reshape(output, (OFMDim_x, IFMChannels*ConvKernelDim_x))
print(y_produced)

print(np.array_equal(y_expected, y_produced))


[[  1   2   3   4   5   6   7  21  22  23  24  25  26  27  41  42  43  44
   45  46  47  61  62  63  64  65  66  67  81  82  83  84  85  86  87 101
  102 103 104 105 106 107 121 122 123 124 125 126 127 141 142 143 144 145
  146 147 161 162 163 164 165 166 167 181 182 183 184 185 186 187]
 [  2   3   4   5   6   7   8  22  23  24  25  26  27  28  42  43  44  45
   46  47  48  62  63  64  65  66  67  68  82  83  84  85  86  87  88 102
  103 104 105 106 107 108 122 123 124 125 126 127 128 142 143 144 145 146
  147 148 162 163 164 165 166 167 168 182 183 184 185 186 187 188]
 [  3   4   5   6   7   8   9  23  24  25  26  27  28  29  43  44  45  46
   47  48  49  63  64  65  66  67  68  69  83  84  85  86  87  88  89 103
  104 105 106 107 108 109 123 124 125 126 127 128 129 143 144 145 146 147
  148 149 163 164 165 166 167 168 169 183 184 185 186 187 188 189]
 [  4   5   6   7   8   9  10  24  25  26  27  28  29  30  44  45  46  47
   48  49  50  64  65  66  67  68  69  70  84  85  86  87  